In [8]:
import pandas as pd
#df = pd.read_excel('https://ncses.nsf.gov/pubs/nsf19301/data/sed17-sr-tab024.xlsx')
import requests
import xlrd

example_url = 'https://ncsesdata.nsf.gov/ids/sed'
r = requests.get(example_url)  # make an HTTP request

workbook = xlrd.open_workbook(file_contents=r.content)  # open workbook
worksheet = workbook.sheet_by_index(0)  # get first sheet
first_row = worksheet.row(0)  # you can iterate over rows of a worksheet as well

print(first_row)  # list of cells

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'<!DOCTYP'

In [7]:
import requests
import xlrd

example_url = 'http://www.excel-easy.com/examples/excel-files/fibonacci-sequence.xlsx'
r = requests.get(example_url)  # make an HTTP request

workbook = xlrd.open_workbook(file_contents=r.content)  # open workbook
worksheet = workbook.sheet_by_index(0)  # get first sheet
first_row = worksheet.row(0)  # you can iterate over rows of a worksheet as well

print(first_row)  # list of cells

[number:0.0]


In [ ]:
import urllib
import requests

In [ ]:
dls = "https://ncses.nsf.gov/pubs/nsf20311#&/nsf20311-tab002"
urllib.urlretrieve(dls, "test.xls")

In [ ]:
import requests
dls = "https://ncses.nsf.gov/pubs/nsf20311#&/nsf20311-tab002.xlsx"
resp = requests.get(dls)

output = open('test.xls', 'wb')
output.write(resp.content)
output.close()

In [ ]:
type(output)

In [1]:
import io
import requests, zipfile  #, StringIO

# r = requests.get("https://ncsesdata.nsf.gov/datadownload/psd17Public.zip", stream=True)
# z = zipfile.ZipFile(io.StringIO(r.content))
# z.extractall()
import requests, zipfile, io
r = requests.get("https://www.nsf.gov/statistics/ffrdc/pub_data.cfm/data/exp2018.xlsx.zip", stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

BadZipFile: File is not a zip file

In [ ]:
url="https://raw.githubusercontent.com/cs109/2014_data/master/countries.csv"
c=pd.read_csv(url)

In [ ]:
c.columns

In [ ]:
c.head()